# 04 大语言模型
大型语言模型（LLM）是LangChain的核心组件。LangChain不提供自己的LLM，而是提供了一个标准接口，用于与许多不同的LLM进行交互。
https://python.langchain.com/docs/modules/model_io/models/llms/


In [2]:
#设置代理
import os
os.environ['http_proxy'] = 'http://127.0.0.1:10809'
os.environ['https_proxy'] = 'http://127.0.0.1:10809'

In [5]:
from langchain.llms import OpenAI
llm = OpenAI()

使用 LLM 的最简单方法是可调用对象：传入字符串，获取字符串完成。
```shell
__call__: string in -> string out
```
这是他内部函数

In [8]:
#可以直接调用
llm("给我讲一个笑话")

'\n\n两个人在一起聊天，一个问另一个："你最近怎么样？"\n\n另一个回答："我想像一根螺丝钉一样，拧得越来越紧！"'

generate: 批量调用，输出更丰富

generate 允许您使用字符串列表调用模型，从而返回比文本更完整的响应。
此完整响应可能包括多个热门响应和其他特定于LLM提供程序的信息：

In [9]:
llm_result = llm.generate(["给我讲个笑话", "给我讲个诗词"]*15)

In [10]:
len(llm_result.generations)

30

In [11]:
llm_result.generations[0]

[Generation(text='\n\n笑话：\n\n一个男孩走进一家商店，问老板：“你有什么新鲜的东西卖吗？”\n\n老板回答：“有，我有一只新鲜的鸡！”\n\n男孩说：“太好了，我要买。你知道怎么把它变成鸭子吗？”\n\n老板说：“不知道，为什么？”\n\n男孩说：“因为我想给它一个惊喜！”', generation_info={'finish_reason': 'stop', 'logprobs': None})]

In [12]:
llm_result.generations[1]

[Generation(text='\n\n《春晓》\n\n春眠不觉晓，\n处处闻啼鸟。\n夜来风雨声，\n花落知多少。', generation_info={'finish_reason': 'stop', 'logprobs': None})]

In [13]:
llm_result.generations[2]

[Generation(text='\n\n一个猴子去拜访朋友，准备给他带礼物，但是他不知道该带什么，于是他就想：“现在是冬天，我可以带一件厚厚的毛衣给他！”于是他就带了毛衣去拜访朋友，朋友很开心，问猴子：“你怎么知道我需要一件毛衣呢？”猴子说：“因为这是猴子送人的冬衣！”', generation_info={'finish_reason': 'stop', 'logprobs': None})]

In [14]:
llm_result.generations[3]

[Generation(text='\n\n春晓\n\n孟浩然\n\n春眠不觉晓，\n\n处处闻啼鸟。\n\n夜来风雨声，\n\n花落知多少。', generation_info={'finish_reason': 'stop', 'logprobs': None})]

In [15]:
#您还可以访问返回的提供程序特定信息。此信息在提供商之间没有标准化。
llm_result.llm_output

{'token_usage': {'total_tokens': 4236,
  'prompt_tokens': 435,
  'completion_tokens': 3801},
 'model_name': 'text-davinci-003'}

## 4.1 异步接口
LangChain通过利用asyncio库为LLM提供异步支持。
异步支持对于同时调用多个 LLM 特别有用，因为这些调用是网络绑定的。
目前、 OpenAI PromptLayerOpenAI ChatOpenAI 、 Anthropic 和 Cohere 受支持，但对其他 LLM 的异步支持已在路线图上。
您可以使用该方法 agenerate 异步调用 OpenAI LLM。


In [4]:
import openai
openai.proxy = os.getenv('https_proxy')

In [5]:
# 导入所需的模块
import time  # 用于计时
import asyncio  # 用于处理异步编程

from langchain.llms import OpenAI  # 从langchain.llms库导入OpenAI类

# 定义一个串行（同步）方式生成文本的函数
def generate_serially():
    llm = OpenAI(temperature=0.9)  # 创建OpenAI对象，并设置temperature参数为0.9
    for _ in range(10):  # 循环10次
        resp = llm.generate(["Hello, how are you?"])  # 调用generate方法生成文本
        print(resp.generations[0][0].text)  # 打印生成的文本

# 定义一个异步生成文本的函数
async def async_generate(llm):
    resp = await llm.agenerate(["Hello, how are you?"])  # 异步调用agenerate方法生成文本
    print(resp.generations[0][0].text)  # 打印生成的文本

# 定义一个并发（异步）方式生成文本的函数
async def generate_concurrently():
    llm = OpenAI(temperature=0.9)  # 创建OpenAI对象，并设置temperature参数为0.9
    tasks = [async_generate(llm) for _ in range(10)]  # 创建10个异步任务
    await asyncio.gather(*tasks)  # 使用asyncio.gather等待所有异步任务完成




In [6]:
# 记录当前时间点
s = time.perf_counter()
# 使用异步方式并发执行生成文本的任务
# 如果在Jupyter以外运行此代码，使用 asyncio.run(generate_concurrently())
await generate_concurrently()
# 计算并发执行所花费的时间
elapsed = time.perf_counter() - s
print("\033[1m" + f"Concurrent executed in {elapsed:0.2f} seconds." + "\033[0m")




I'm doing well, thank you. How about you?

I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, how about you?


I'm doing well, thank you. How about you?


I'm doing good, thanks! How about you?


I'm doing well, thank you for asking! How about yourself?


I'm doing well, thank you. How about you?


I'm doing well, thanks for asking! How about you?


I'm doing well, thank you. How about you?
Concurrent executed in 2.14 seconds.


In [19]:
# 记录当前时间点
s = time.perf_counter()
# 使用同步方式串行执行生成文本的任务
generate_serially()
# 计算串行执行所花费的时间
elapsed = time.perf_counter() - s
print("\033[1m" + f"Serial executed in {elapsed:0.2f} seconds." + "\033[0m")


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm doing well, thank you. How about you?


I'm good, thanks! How about you?


I'm doing well. How about you?


I'm doing great. How about you?


I'm doing well, thank you. How about you?
Serial executed in 12.38 seconds.


## 4.2 定制大语言模型
如果您想使用自己的 LLM 或与 LangChain 中支持的包装器不同的包装器。
自定义LLM只需要实现一件必需的事情：

一个 _call 方法，它接受一个字符串、一些可选的非索引字，并返回一个字符串

它可以实现第二个可选的东西：

用于帮助打印此类 _identifying_params 的属性。应该返回字典。

In [11]:
# 让我们实现一个非常简单的自定义 LLM，它只返回输入的前 N 个字符。
from typing import Any, List, Mapping, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM

#这个类 CustomLLM 继承了 LLM 类，并增加了一个新的类变量 n。
#有两个 property 装饰的方法，分别是 _llm_type 和 _identifying_params，这两个方法都返回一些固定的属性值。
#_call 方法主要是对输入的 prompt 字符串进行处理，返回前 n 个字符。如果提供了 stop 参数，它将引发一个异常。


# 继承自 LLM 的 CustomLLM 类
class CustomLLM1(LLM):

    # 类变量，表示一个整数
    n: int

    # 一个属性装饰器，用于获取 _llm_type 的值
    @property
    def _llm_type(self) -> str:
        # 返回 "custom" 字符串作为 _llm_type 的值
        return "custom"

    # _call 方法用于处理某些操作
    def _call(
        self,
        prompt: str,  # 输入的提示字符串
        stop: Optional[List[str]] = None,  # 可选的停止字符串列表，默认为 None
        run_manager: Optional[CallbackManagerForLLMRun] = None,  # 可选的回调管理器，默认为 None
    ) -> str:
        # 如果 stop 参数不为 None，则抛出 ValueError 异常
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        # 返回 prompt 字符串的前 n 个字符
        return prompt[: self.n]

    # 一个属性装饰器，用于获取 _identifying_params 的值
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        # 这个方法的文档字符串，说明这个方法的功能是获取标识参数
        """Get the identifying parameters."""
        # 返回一个字典，包含 n 的值
        return {"n": self.n}


In [12]:
llm = CustomLLM1(n=10)

In [13]:
llm("This is a foobar thing")

'This is a '

In [14]:
print(llm)

CustomLLM1
Params: {'n': 10}
